We are interested in the swarm passes over our event. To do this we need to:

A: Plot the Images

B: Get the emphermis data for swarm B

C: Find the footprint of swarm B at the assumed emission height of 110km

D: Overplot the emphermis data with the ground based observations

E: Overplot the velocity quivers with ground based observations

Lets first grab our images from the package asilib

Lets choose a 3x2 grid to plot the images, our images will start at 14:04.20 and then move 10 seconds

In [ ]:
import numpy as np #Lets handle our imports now for section A
import matplotlib.pyplot as plt
import asilib
import asilib.asi
from datetime import datetime, timedelta
from scipy.optimize import curve_fit
import scienceplots

plt.style.use(['science', 'no-latex'])


We want a 3x3 pannel with 10 second intervals at altitude of 150km

In [ ]:
start_time= datetime(2022,12,19,14,4,20)
time_array= [start_time + timedelta(seconds=10*x) for x in range(9)]
alt=150

Part A:

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3, dpi=300)
axlist=axes.flatten()
plt.axis('off')

for i in range(len(time_array)):
    asi=asilib.asi.trex_rgb(location_code='yknf', alt=alt, time=time_array[i])

    asi.plot_fisheye(ax=axlist[i], label=False, cardinal_directions='NEWS')
    axlist[i].set_axis_off()
    axlist[i].set_title(time_array[i].strftime('%H:%M:%S%f')[:-6])
plt.tight_layout(pad=0.1, w_pad=0.6, h_pad=-1)
plt.savefig("test.png",bbox_inches='tight')

Now lets move onto part B

We are going to use the viresclient ESA client to download swarm empharmsis data with a frequency that matches the cadence of 3 so that it matches with the cadence of trex rgb.

In [ ]:
from viresclient import set_token
from viresclient import SwarmRequest
import geopack.geopack as gp



In [ ]:
#Now we need to set our token, for my case to protect my token I import from a file not in the github cloud, however you can just paste your code from viresclient, just don't share it publically!


In [ ]:
#From documentation link
def requester(sc_collection, measurement, residual, sampling_step=None, **kwargs):
    try:
        request = SwarmRequest()
        request.set_collection(sc_collection)
        if residual == True:
            request.set_products(
                measurements=measurement,
                models=["CHAOS"],
                residuals=True,
                sampling_step=sampling_step,
            )
        else:
            request.set_products(
                measurements=measurement,
                models=["CHAOS"],
                sampling_step=sampling_step,
            )
        data = request.get_between(time_array[0], time_array[-1]+timedelta(seconds=10), **kwargs) #sets to get data between the first and last value in time array iniatialized earlier
        df = data.as_dataframe()
    except:
        df = []
    return df

In [ ]:
ds = requester( 
    "SW_OPER_MAGB_LR_1B", #Mag B, low resolution, 1Hz B (Magnetic field)
    "B_NEC", #Magnetic field in NEC coordinates
    True, 
    asynchronous=False,
    show_progress=False,
    sampling_step="PT{}S".format(10)) #cadence of 10 to match our images, in practice this selects every 10th data point since the resolution is 1 sample a second (sps)



In [ ]:
latitude, longitude, altitude = ds['Latitude'].to_numpy(), ds['Longitude'].to_numpy(),  (ds["Radius"].to_numpy()-6.371e6)/1e3 #km  # Gets Emphermis data
print(ds.index)

Good now that we have empharsis we need to find the footprint. To do this, we use geopack and asilib. Alternatively if you have IRBEM installed you can use asilib.conjunction.footprint to do this automagically.

In [ ]:
t1 = time_array[0]
t0 = datetime(1970,1,1)
ut = (t1-t0).total_seconds()
lat_sat=np.deg2rad(latitude)
lon_sat=np.deg2rad(longitude)
gp.recalc(ut)
print(altitude)
r, theta= gp.geodgeo(altitude,lat_sat,1) #TODO magically, r is 10km less than if you calculated r manually, is this real

x_gc,y_gc,z_gc = gp.sphcar((r)/6371,theta,lon_sat,1)  #spherical to cartesian
 

x_gsm, y_gsm, z_gsm = gp.geogsm(x_gc,y_gc,z_gc, 1) #cartesian to gsm

x_foot,y_foot,z_foot=np.zeros(len(x_gsm)), np.zeros(len(y_gsm)), np.zeros(len(z_gsm)) #initalize an array


In [ ]:
for index in range(len(x_gsm)):
    x_foot_int, y_foot_int, z_foot_int, xx2, yy2,zz2 = gp.trace(x_gsm[index], y_gsm[index], z_gsm[index], dir=1,rlim=2, r0=(alt-10+6371)/6371, maxloop=300 ) #traces each set of lat,lon,alt outward
    def curve_fit_func():
        def cubic(t, a, b, c, d):
            return a*t**3 + b*t**2 + c*t + d
        r = np.linspace(1, 1.5, 100000)

        radius_data=np.sqrt(xx2**2+yy2**2+zz2**2)

        params_x, _ = curve_fit(cubic, radius_data, xx2) #Constructs fits on the traces inward since the spatial resolution produced by geopack is limited.
        params_y, _ = curve_fit(cubic, radius_data, yy2)
        params_z, _ = curve_fit(cubic, radius_data, zz2)

        def x(t):
            return cubic(t, *params_x)

        def y(t):
            return cubic(t, *params_y)

        def z(t):
            return cubic(t, *params_z)
        def radius(t):
            return np.sqrt(x(t)**2 + y(t)**2 + z(t)**2)

        index_closest=np.argmin(np.abs(radius(r)-(alt-10+6371)/6371))

        return x(r[index_closest]),y(r[index_closest]),z(r[index_closest])

    x_foot[index],y_foot[index],z_foot[index] = curve_fit_func()

x_done, y_done, z_done = gp.geogsm(x_foot, y_foot, z_foot, -1)

alt_sat_done, lat_sat_done,lon_sat_done = np.zeros(len(x_done)), np.zeros(len(x_done)), np.zeros(len(x_done))
for index in range(len(x_done)):
    
    r_done,theta_done,lon_sat_done[index]= gp.sphcar(x_done[index], y_done[index], z_done[index],-1)

    alt_sat_done[index], lat_sat_done[index]= gp.geodgeo(r_done*6371,theta_done,-1) #TODO check if this is right

print(alt_sat_done, 'altitude derived from fit')

if np.any(np.abs(alt_sat_done - alt) > 5):
    raise Exception("One or more values in the footprinting are greater than 5km away from the specified alt. Contact owner for a fix, not your fault")
print(np.rad2deg(lon_sat_done)-360,np.rad2deg(lat_sat_done) , 'lat and lon' )
sat_lla=np.array([np.rad2deg(lat_sat_done), np.rad2deg(lon_sat_done)-360, alt_sat_done]).T

conjunction_obj = asilib.Conjunction(asi, (np.array(time_array), sat_lla))

Now lets use asilib to synthsize the empharsis data with the images

In [ ]:
sat_azel, sat_azel_pixels = conjunction_obj.map_azel()

In [ ]:
plt.rcParams["axes.edgecolor"] = "0.15"
plt.rcParams["axes.linewidth"]  = 2

In [ ]:
fig, axes = plt.subplots(figsize=(10,8), nrows=3, ncols=3, dpi=300)
axlist=axes.flatten()
plt.axis('off')
plt.tight_layout()
for i in range(len(time_array)):
    asi=asilib.asi.trex_rgb(location_code='yknf', alt=alt, time=time_array[i])

    asi.plot_fisheye(ax=axlist[i], label=False, cardinal_directions='NW')
    axlist[i].set_axis_off()
    axlist[i].set_title(time_array[i].strftime('%H:%M:%S%f')[:-6], fontsize=18)
    print(time_array[i].strftime('%H:%M:%S%f')[:-6],  time_array[i])
print(sat_azel_pixels[:, 1], time_array)
for i in range(len(axlist)):
    axlist[i].plot(sat_azel_pixels[:, 0],
                            sat_azel_pixels[:, 1], color='red', linestyle="dashed")
    axlist[i].scatter(sat_azel_pixels[i, 0], sat_azel_pixels[i, 1],
                marker='o', s=5, color='red')
plt.subplots_adjust(hspace=0.15, wspace=-0.2)  # Adjust the value as needed

In [ ]:
fig, axes = plt.subplots(figsize=(10,8), nrows=3, ncols=3, dpi=300)
axlist=axes.flatten()

for i in range(len(time_array)):
    asi=asilib.asi.trex_rgb(location_code='yknf', alt=alt, time=time_array[i], colors='rgb')

    asi.plot_map(ax=axlist[i], asi_label=False)
    axlist[i].set_ylim(59, 66)
    axlist[i].set_title(time_array[i].strftime('%H:%M:%S%f')[:-6])
    axlist[i].plot(sat_lla[:,1], sat_lla[:,0], color='red', linestyle='dashed')
    axlist[i].scatter(sat_lla[i,1], sat_lla[i,0], color='blue')
plt.savefig("test.png",bbox_inches='tight')


In [ ]:
fig, axes = plt.subplots(figsize=(10,8), nrows=3, ncols=3, dpi=300, sharex=True, sharey=True, tight_layout=True)

axlist=axes.flatten()

for i in range(len(time_array)):
    axlist[i].tick_params(axis='both', colors='white', labelcolor='black')
    asi=asilib.asi.trex_rgb(location_code='yknf', alt=alt, time=time_array[i], colors='rgb')

    asi.plot_map(ax=axlist[i], asi_label=False)
    axlist[i].set_title(time_array[i].strftime('%H:%M:%S%f')[:-6])
    axlist[i].set_xlim(-119,-111)
    axlist[i].set_ylim(61.75,63)
    axlist[i].plot(sat_lla[:,1], sat_lla[:,0], color='white', linestyle='dashed')
    axlist[i].scatter(sat_lla[i,1], sat_lla[i,0], color='red')
plt.savefig("test.png")
fig.suptitle("Track of Swarm B Over Auroral Vortices")
fig.supxlabel("Geographic Longitude", y=0.02)
fig.supylabel("Geographic Latitude")



Now we want to had velocity quivers to our plot for added context

In [ ]:
#From documentation link
def requester(sc_collection, measurement, residual, sampling_step=None, **kwargs):
    try:
        request = SwarmRequest()
        request.set_collection(sc_collection)
        if residual == True:
            request.set_products(
                measurements=measurement,
                models=["CHAOS"],
                residuals=True,
                sampling_step=sampling_step,
            )
        else:
            request.set_products(
                measurements=measurement,
                models=["CHAOS"],
                sampling_step=sampling_step,
            )
        data = request.get_between(time_array[0], time_array[-1], **kwargs) #sets to get data between the first and last value in time array iniatialized earlier
        df = data.as_dataframe()
    except:
        df = []
    return df

In [ ]:
measurements_E = [
        "VsatN",
        "VsatE",
        "VsatC",
        "Evx",
        "Evy",
        "Evz",
        "Vixv",
        "Viy",
        "Viz",
        "Quality_flags",
    ]
dsE = requester( 
    "SW_EXPT_EFIB_TCT16", #Mag B, high resolution, 50Hz B (Magnetic field)
    measurements_E, #Magnetic field in NEC coordinates
    True, 
    asynchronous=False,
    show_progress=False) 
dsB = requester( 
    'SW_OPER_MAGB_HR_1B', #Mag B, high resolution, 50Hz B (Magnetic field)
    ["q_NEC_CRF"], #Magnetic field in NEC coordinates
    False, 
    asynchronous=False,
    show_progress=False)
Etime = dsE["Viy"].index.to_numpy()
Btime = dsB['Latitude'].index.to_numpy()
latitude_B, longitude_B, altitude_B = dsB['Latitude'].to_numpy(), dsB['Longitude'].to_numpy(),  (dsB["Radius"].to_numpy()-6.371e6)/1e3 #km  # Gets Emphermis data


In [ ]:

from scipy.optimize import curve_fit
import geopack.geopack as gp
def footprint(time, latitude, longitude, altitude, alt,vsw= [-400,0,0]):
    """
    time, datetime, time for magnetic data for footprint
    vsw velocity of solar wind, tuple of x,y,z
    longitude of satellite in degrees
    latitude of satellite in degrees
    altitude of satellite in km from centre of earth (should be above ~6371)
    THIS CODE ONLY Works in the NOrthern hemisphere

    """
    def cubic(t, a, b, c, d):
            return a*t**3 + b*t**2 + c*t + d
    def x(t, params_x):
        return cubic(t, *params_x)

    def y(t, params_y):
        return cubic(t, *params_y)

    def z(t, params_z):
        return cubic(t, *params_z)
    def radius(t, params_x, params_y, params_z):
        return np.sqrt(x(t, params_x)**2 + y(t, params_y)**2 + z(t, params_z)**2)
    
    def curve_fit_func(xx,yy,zz, differencealt):
        
        r = np.linspace(1, 1.5, 10000)# construct an array of radiuses from 1-1.5

        radius_data=np.sqrt(xx**2+yy**2+zz**2)

        params_x, _ = curve_fit(cubic, radius_data, xx) #Constructs fits on the traces inward since the spatial resolution produced by geopack is limited.
        params_y, _ = curve_fit(cubic, radius_data, yy)
        params_z, _ = curve_fit(cubic, radius_data, zz)

        

        index_closest=np.argmin(np.abs(radius(r, params_x, params_y, params_z)-(alt-differencealt+6371)/6371))#Find the index that produces the closest radius to the altitude

        return x(r[index_closest],params_x ),y(r[index_closest],params_y ),z(r[index_closest],params_z )
    
    t1 = time
    t0 = datetime(1970,1,1) #epoch
    ut = (t1-t0).total_seconds()
    lat_sat=np.deg2rad(latitude)
    lon_sat=np.deg2rad(longitude) #converts to radii
    gp.recalc(ut)
    r, theta= gp.geodgeo(altitude,lat_sat,1) #this r accounts for earths oblateness, so we need to find the difference between my 6371 assumption and the real value and account for that
    differencearray= (altitude+6371)-r
    x_gc,y_gc,z_gc = gp.sphcar((r)/6371,theta,lon_sat,1)  #spherical to cartesian
    

    x_gsm, y_gsm, z_gsm = gp.geogsm(x_gc,y_gc,z_gc, 1) #cartesian to gsm

    x_foot,y_foot,z_foot=np.zeros(len(x_gsm)), np.zeros(len(y_gsm)), np.zeros(len(z_gsm)) #initalize an array
    for index in range(len(x_gsm)):
        x_foot_int, y_foot_int, z_foot_int, xx2, yy2,zz2 = gp.trace(x_gsm[index], y_gsm[index], z_gsm[index], dir=1,rlim=3, maxloop=1000 ) #traces each set of lat,lon,alt outward


        x_foot[index],y_foot[index],z_foot[index] = curve_fit_func(xx2,yy2,zz2, differencearray[index])


            

    x_done, y_done, z_done = gp.geogsm(x_foot, y_foot, z_foot, -1)

    alt_sat_done, lat_sat_done,lon_sat_done = np.zeros(len(x_done)), np.zeros(len(x_done)), np.zeros(len(x_done))
    for index in range(len(x_done)):
        
        r_done,theta_done,lon_sat_done[index]= gp.sphcar(x_done[index], y_done[index], z_done[index],-1)

        alt_sat_done[index], lat_sat_done[index]= gp.geodgeo(r_done*6371,theta_done,-1) #TODO check if this is right

    print(alt_sat_done, 'altitude derived from fit')

    if np.any(np.abs(alt_sat_done - alt) > 5):
        raise Exception("One or more values in the footprinting are greater than 5km away from the specified alt. Contact owner for a fix, not your fault")
    print(np.rad2deg(lon_sat_done)-360,np.rad2deg(lat_sat_done) , 'lat and lon' )
    sat_lla=np.array([ np.rad2deg(lat_sat_done), np.rad2deg(lon_sat_done)-360,  alt_sat_done])
    return sat_lla


def find_closest_indices(times1, times2):
    # Convert to numpy arrays
    times1 = np.array(times1)
    times2 = np.array(times2)
    
    # Compute the differences between each time in times1 and all times in times2
    # Resulting in a 2D array where each row contains the absolute differences for one time in times1
    differences = np.abs(times1[:, None] - times2)
    
    # Find the index of the minimum difference for each time in times1
    closest_indices = np.argmin(differences, axis=1)
    
    return closest_indices

from scipy.spatial.transform import Rotation as R
import numpy as np

def quaternion_inverse_scipy(q):
    # Ensure q is a numpy array
    q = np.asarray(q)
    
    # Create a Rotation object from the quaternion
    rotation = R.from_quat(q)  # Note: scipy uses [x, y, z, w] format
    
    # Compute the inverse rotation
    inverse_rotation = rotation.inv()
    
    
    return inverse_rotation

In [ ]:
sat_lla_B=footprint(time_array[0], latitude_B, longitude_B, altitude_B, alt, vsw=[-345,12,-12])
#This can take 10+ minutes, sry but I wanted to make my own pythonic-based footprinter


In [ ]:
indicies=find_closest_indices(dsE.index, dsB.index)
quatnecrf=dsB["q_NEC_CRF"].to_numpy()[indicies]
quaternions = []
vsat=np.array([dsE["Vixv"] , dsE["Viy"], dsE["Viz"]]).T
Etime = dsE.index
ENEC=[]
VNEC=[]
for i in range(len(quatnecrf)):
    inverse_quat = quaternion_inverse_scipy(dsB["q_NEC_CRF"].to_numpy()[indicies][i])

    rot_NEC_V= inverse_quat.apply(vsat[i])
    VNEC.append(rot_NEC_V)

ENEC=np.array(ENEC)
VNEC=np.array(VNEC)

In [ ]:
from scipy.interpolate import CubicSpline
cs_lat = CubicSpline( Btime, sat_lla_B[0])
cs_lon = CubicSpline( Btime, sat_lla_B[1])
cs_alt = CubicSpline( Btime, sat_lla_B[2])

#sat_lla_E=footprint(time_array[0], latitude_E, longitude_E, altitude_E, alt, vsw=[-345,12,-12])
sat_lla_E= np.array([cs_lat(Etime), cs_lon(Etime),  cs_alt(Etime)]) #km  # Gets Emphermis data latitude, longitude, altitude


In [ ]:
U_perpendicular = -VNEC[:, 1] / 2000  # Inverting y-component for perpendicular direction
V_perpendicular = VNEC[:, 1] / 2000   # Keeping the x-component
u_zeros = np.zeros_like(VNEC[:, 1])
QV1= plt.quiver( Etime,sat_lla_E[0], V_perpendicular, u_zeros, color='white', scale=1, scale_units='inches', label="Eastward Velocity")

In [ ]:
fig, axes = plt.subplots(figsize=(10,8), nrows=3, ncols=3, dpi=300, sharex=True, sharey=True, tight_layout=True)

axlist=axes.flatten()

for i in range(len(time_array)):
    axlist[i].tick_params(axis='both', colors='white', labelcolor='black', which='both')
    asi=asilib.asi.trex_rgb(location_code='yknf', alt=alt, time=time_array[i], colors='rgb')

    asi.plot_map(ax=axlist[i], asi_label=False)
    axlist[i].set_title(time_array[i].strftime('%H:%M:%S%f')[:-6])
    axlist[i].set_xlim(-119,-109)
    axlist[i].set_ylim(61.75,63)
    axlist[i].quiver(sat_lla_E[1], sat_lla_E[0],V_perpendicular, u_zeros, color='white', scale=1.9, scale_units='inches', label="Eastward Velocity", alpha=0.2)
    indicies= np.where((Etime >= np.datetime64(time_array[i] -  timedelta(seconds=6))) & (Etime <= np.datetime64(time_array[i] + timedelta(seconds=6))))[0]
    axlist[i].quiver(sat_lla_E[1][indicies], sat_lla_E[0][indicies],V_perpendicular[indicies], u_zeros[indicies], color='red', scale=1.90, scale_units='inches', label="Eastward Velocity", alpha=0.4)
    middle_index=np.argmin(np.abs(Etime -time_array[i]))
    axlist[i].scatter(sat_lla_E[1][middle_index], sat_lla_E[0][middle_index], color='white', s=25)
plt.savefig("test.png")
fig.suptitle("Track of Swarm B Over Auroral Vortices")
fig.supxlabel("Geographic Longitude", y=0.02)
fig.supylabel("Geographic Latitude")

